In [1]:
import pandas as pd
import warnings
from IPython.utils import io
import sys
import numpy as np
from functools import reduce

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

stars_dir = '~/GitHub/stars-data-builder/'
hos_dir = '~/Desktop/Rush/CMS_HospitalArchives/'


# 
----

# Find areas of accuracy

The overall accuracy of estimates made in July 2023 was 71%. Perhaps, the accuracy was concentrated among particular hospitals.

# Load derived input data files for 2024

In [2]:
jul_input_df = pd.read_csv(stars_dir + "Reproduce_Stars_Input/2024/Input_File/data_for_2024_prognostications_from_Jul2023.csv")
ls = []
for p in jul_input_df['PROVIDER_ID'].tolist():
    p = int(p)
    p = str(p)
    if '666666' in p:
        p = p[:-6]
        p = p + 'F'
    while len(p) < 6:
        p = '0' + p
    ls.append(p)

jul_input_df['PROVIDER_ID'] = ls

with io.capture_output() as captured: 
    input_df_2024 = pd.read_sas(stars_dir + '2024/2024-07 Stars Release/alldata_2024jul.sas7bdat')
input_df_2024 = input_df_2024.applymap(lambda x: x.decode() if isinstance(x, bytes) else x)

print(input_df_2024.shape)
prvdrs = input_df_2024['PROVIDER_ID'].unique()
print(len(prvdrs), 'hospitals in 2024')

input_df_2024.head()

(4658, 95)
4658 hospitals in 2024


,PROVIDER_ID,HAI_1_DEN_VOL,HAI_2_DEN_VOL,HAI_3_DEN_VOL,HAI_4_DEN_VOL,HAI_5_DEN_VOL,HAI_6_DEN_VOL,HAI_1_DEN_PRED,HAI_2_DEN_PRED,HAI_3_DEN_PRED,HAI_4_DEN_PRED,HAI_5_DEN_PRED,HAI_6_DEN_PRED,HAI_1,HAI_2,HAI_3,HAI_4,HAI_5,HAI_6,READM_30_HOSP_WIDE,READM_30_HIP_KNEE,EDAC_30_HF,READM_30_COPD,EDAC_30_AMI,EDAC_30_PN,MORT_30_STK,MORT_30_PN,MORT_30_HF,MORT_30_COPD,MORT_30_AMI,COMP_HIP_KNEE,READM_30_HOSP_WIDE_DEN,READM_30_HIP_KNEE_DEN,EDAC_30_HF_DEN,READM_30_COPD_DEN,EDAC_30_AMI_DEN,EDAC_30_PN_DEN,MORT_30_STK_DEN,MORT_30_PN_DEN,MORT_30_HF_DEN,MORT_30_COPD_DEN,MORT_30_AMI_DEN,COMP_HIP_KNEE_DEN,OP_2,OP_2_DEN,OP_3B,OP_3B_DEN,OP_8,OP_8_DEN,OP_10,OP_10_DEN,OP_13,OP_13_DEN,OP_18B,OP_18B_DEN,OP_22,OP_22_DEN,OP_23,OP_23_DEN,OP_29,OP_29_DEN,PSI_4_SURG_COMP,PSI_4_SURG_COMP_DEN,PSI_90_SAFETY,IMM_3_DEN,IMM_3,HCP_COVID_19_DEN,HCP_COVID_19,PC_01,PC_01_DEN,SEP_1,SEP_1_DEN,H_RESP_RATE_P,H_COMP_1_STAR_RATING,H_COMP_2_STAR_RATING,H_COMP_3_STAR_RATING,H_COMP_5_STAR_RATING,H_COMP_6_STAR_RATING,H_COMP_7_STAR_RATING,H_GLOB_STAR_RATING,H_INDI_STAR_RATING,H_NUMB_COMP,PSI_90_SAFETY_DEN,MORT_30_CABG,MORT_30_CABG_DEN,READM_30_CABG,READM_30_CABG_DEN,OP_32,OP_32_DEN,OP_35_ADM,OP_35_ADM_DEN,OP_35_ED,OP_35_ED_DEN,OP_36,OP_36_DEN
0,010001,9149.0,17310.0,214.0,NaN,104733.0,104733.0,9.597,24.766,5.994,NaN,11.400,67.066,0.938,0.363,1.335,NaN,0.965,0.507,0.142,0.038,23.4,0.190,-15.4,23.6,0.148,0.180,0.089,0.088,0.120,0.027,2912.0,49.0,614.0,117.0,274.0,403.0,398.0,400.0,549.0,107.0,278.0,49.0,NaN,NaN,NaN,NaN,0.380,79.0,0.061,1410.0,0.028,178.0,214.0,348.0,0.05,52960.0,NaN,NaN,0.47,17.0,184.68,134.0,1.21,3905.0,0.95,2496.0,0.836,0.00,32.0,0.65,127.0,15.0,3.0,3.0,3.0,3.0,4.0,4.0,3.5,3.5,544.0,2542.038500,0.041,132.0,0.105,126.0,12.9,170.0,11.9,202.0,4.9,202.0,1.1,668.0
1,010005,3194.0,8277.0,96.0,NaN,36794.0,34887.0,1.989,4.019,2.626,NaN,1.847,10.066,2.514,0.995,0.762,NaN,0.541,0.497,0.128,0.034,22.1,0.166,NaN,-6.2,0.153,0.233,0.149,0.099,0.136,0.023,1052.0,172.0,129.0,136.0,NaN,285.0,81.0,289.0,121.0,126.0,27.0,155.0,NaN,NaN,57.0,16.0,0.477,130.0,0.120,1057.0,0.042,189.0,145.0,1074.0,0.03,56820.0,0.58,12.0,0.96,180.0,183.49,43.0,0.97,2700.0,0.80,2552.0,0.807,0.02,200.0,0.69,252.0,18.0,3.0,4.0,1.0,3.0,4.0,3.0,3.0,3.0,824.0,978.028994,NaN,NaN,NaN,NaN,14.2,739.0,7.9,107.0,5.5,107.0,1.9,406.0
2,010006,5343.0,8715.0,111.0,NaN,63727.0,60304.0,5.801,11.166,2.950,NaN,5.283,27.805,0.172,0.358,0.000,NaN,1.514,0.072,0.134,0.053,-4.7,0.176,28.1,-0.4,0.172,0.195,0.125,0.099,0.165,0.046,2310.0,138.0,441.0,158.0,273.0,472.0,227.0,469.0,388.0,148.0,254.0,145.0,NaN,NaN,NaN,NaN,0.462,39.0,0.101,978.0,0.045,221.0,168.0,360.0,0.01,42286.0,0.75,16.0,0.85,82.0,173.63,96.0,1.17,2536.0,0.67,1882.0,0.796,0.04,28.0,0.57,126.0,19.0,2.0,3.0,2.0,2.0,3.0,2.0,2.0,2.5,1503.0,1753.979899,0.036,95.0,0.124,89.0,12.1,1355.0,NaN,NaN,NaN,NaN,1.4,484.0
3,010007,NaN,NaN,NaN,NaN,NaN,5511.0,NaN,NaN,NaN,NaN,NaN,2.660,NaN,NaN,NaN,NaN,NaN,0.376,0.157,0.042,-1.9,0.200,NaN,-9.4,NaN,0.285,0.125,0.137,NaN,NaN,258.0,26.0,31.0,34.0,NaN,72.0,NaN,88.0,26.0,34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.034,146.0,NaN,NaN,132.0,1275.0,0.04,11202.0,NaN,NaN,0.23,111.0,NaN,NaN,0.95,350.0,0.53,252.0,0.601,NaN,NaN,0.93,43.0,24.0,3.0,5.0,4.0,3.0,3.0,3.0,3.5,3.0,189.0,228.286193,NaN,NaN,NaN,NaN,13.4,109.0,NaN,NaN,NaN,NaN,1.2,59.0
4,010008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,85.0,NaN,NaN,116.0,340.0,0.00,6239.0,NaN,NaN,0.67,24.0,NaN,NaN,NaN,126.0,0.45,163.0,0.797,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.9,42.0,NaN,NaN,NaN,NaN,NaN,NaN


## Filter and format the input files

In [3]:
jul_prvdrs = jul_input_df['PROVIDER_ID'].unique().tolist()
act_prvdrs = input_df_2024['PROVIDER_ID'].unique().tolist()

lsts = [jul_prvdrs, act_prvdrs]
for ls in lsts: print(len(ls), 'hospitals')

sets = [set(ls) for ls in lsts]
common_prvdrs = list(sets[0].intersection(*sets[1:]))

print('\n', len(common_prvdrs), 'Common hospitals\n')

input_df_2024 = input_df_2024[input_df_2024['PROVIDER_ID'].isin(common_prvdrs)]

jul_input_df = jul_input_df[jul_input_df['PROVIDER_ID'].isin(common_prvdrs)]
jul_input_df.sort_values(by='PROVIDER_ID', ascending=True, inplace=True)
input_df_2024.sort_values(by='PROVIDER_ID', ascending=True, inplace=True)

jul_input_df['month_year'] = 'July_2023'
input_df_2024['month_year'] = 'Actual'

print('Dataframe shapes')
print(jul_input_df.shape)
print(input_df_2024.shape, '\n')

strs = ['^(?!.*_DEN$).*', '^(?!.*_VOL$).*',
        '^(?!.*_DEN_).*$', '^(?!.*_NUMB_).*$',
        '^(?!OP_2$).*', '^(?!H_RESP_RATE_P$).*']

for st in strs:
    input_df_2024 = input_df_2024.filter(regex=st)
    jul_input_df = jul_input_df.filter(regex=st)
    
print('Dataframe shapes')
print(jul_input_df.shape)
print(input_df_2024.shape, '\n')

jul_input_df.reset_index(drop=True, inplace=True)
input_df_2024.reset_index(drop=True, inplace=True)
input_df_2024 = input_df_2024.merge(jul_input_df, how='outer')

## Combine the input files 

print('Shape of merged DataFrame:', input_df_2024.shape)
input_df_2024.head()

4678 hospitals
4658 hospitals

 4614 Common hospitals

Dataframe shapes
(4614, 96)
(4614, 96) 

Dataframe shapes
(4614, 48)
(4614, 48) 

Shape of merged DataFrame: (9228, 48)


,PROVIDER_ID,HAI_1,HAI_2,HAI_3,HAI_4,HAI_5,HAI_6,READM_30_HOSP_WIDE,READM_30_HIP_KNEE,EDAC_30_HF,READM_30_COPD,EDAC_30_AMI,EDAC_30_PN,MORT_30_STK,MORT_30_PN,MORT_30_HF,MORT_30_COPD,MORT_30_AMI,COMP_HIP_KNEE,OP_3B,OP_8,OP_10,OP_13,OP_18B,OP_22,OP_23,OP_29,PSI_4_SURG_COMP,PSI_90_SAFETY,IMM_3,HCP_COVID_19,PC_01,SEP_1,H_COMP_1_STAR_RATING,H_COMP_2_STAR_RATING,H_COMP_3_STAR_RATING,H_COMP_5_STAR_RATING,H_COMP_6_STAR_RATING,H_COMP_7_STAR_RATING,H_GLOB_STAR_RATING,H_INDI_STAR_RATING,MORT_30_CABG,READM_30_CABG,OP_32,OP_35_ADM,OP_35_ED,OP_36,month_year
0,010001,0.938,0.363,1.335,NaN,0.965,0.507,0.142,0.038,23.4,0.190,-15.4,23.6,0.148,0.180,0.089,0.088,0.120,0.027,NaN,0.380,0.061,0.028,214.0,0.05,NaN,0.47,184.68,1.21,0.95,0.836,0.00,0.65,3.0,3.0,3.0,3.0,4.0,4.0,3.5,3.5,0.041,0.105,12.9,11.9,4.9,1.1,Actual
1,010005,2.514,0.995,0.762,NaN,0.541,0.497,0.128,0.034,22.1,0.166,NaN,-6.2,0.153,0.233,0.149,0.099,0.136,0.023,57.0,0.477,0.120,0.042,145.0,0.03,0.58,0.96,183.49,0.97,0.80,0.807,0.02,0.69,3.0,4.0,1.0,3.0,4.0,3.0,3.0,3.0,NaN,NaN,14.2,7.9,5.5,1.9,Actual
2,010006,0.172,0.358,0.000,NaN,1.514,0.072,0.134,0.053,-4.7,0.176,28.1,-0.4,0.172,0.195,0.125,0.099,0.165,0.046,NaN,0.462,0.101,0.045,168.0,0.01,0.75,0.85,173.63,1.17,0.67,0.796,0.04,0.57,2.0,3.0,2.0,2.0,3.0,2.0,2.0,2.5,0.036,0.124,12.1,NaN,NaN,1.4,Actual
3,010007,NaN,NaN,NaN,NaN,NaN,0.376,0.157,0.042,-1.9,0.200,NaN,-9.4,NaN,0.285,0.125,0.137,NaN,NaN,NaN,NaN,0.034,NaN,132.0,0.04,NaN,0.23,NaN,0.95,0.53,0.601,NaN,0.93,3.0,5.0,4.0,3.0,3.0,3.0,3.5,3.0,NaN,NaN,13.4,NaN,NaN,1.2,Actual
4,010008,NaN,NaN,NaN,NaN,NaN,NaN,0.148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,116.0,0.00,NaN,0.67,NaN,NaN,0.45,0.797,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.9,NaN,NaN,NaN,Actual


#      
------
# Load and merge estimates of star ratings for 2024 with the actual outcomes

In [4]:
stars_df_2024 = pd.read_csv(stars_dir + '2024/2024-07 Stars Release/SAS_CSV_output/CMS_Stars_Jul_2024.csv')
stars_df_2024['month_year'] = 'Actual'

labs = ['July_2023']
for lab in labs:

    tdf = pd.read_csv(stars_dir + 'Reproduce_Stars_Input/2024/SAS_output/CMS_Stars_2024_predictions_from_' + lab + '_data.csv')

    ## Replace the imputed 666666 suffixes of VHA hospitals with their original 'F' suffix
    prvdrs1 = []
    for p in tdf['PROVIDER_ID'].tolist():
        p = str(p)
        if '666666' in p:
            p = p[:-6] + 'F'
        while len(p) < 6:
            p = '0' + p
        prvdrs1.append(p)

    tdf['PROVIDER_ID'] = prvdrs1
    tdf['month_year'] = lab
    stars_df_2024 = stars_df_2024.merge(tdf, how='outer')
    print(stars_df_2024.shape)



(9303, 28)


In [5]:
n_dir = stars_dir + 'CareCompare'
ls = ['PROVIDER_ID', 'Hospital Name', 'State', '2023 overall star rating',
      'Hospital Type', 'Hospital Ownership', 'Emergency Services', 
      'month_year',
     ]

labs = ['July_2023', 'Actual']
flabs = ['07_2023', '01_2024']
for i, lab in enumerate(labs):

    tdf = pd.read_csv(n_dir + '/hospitals_' + flabs[i] + '/Hospital_General_Information.csv')
    tdf['Facility ID'] = tdf['Facility ID'].astype(str)
    
    cols1 = ['Facility ID', 'Facility Name', 'Hospital overall rating']
    cols2 = ['PROVIDER_ID', 'Hospital Name', '2023 overall star rating']

    for i, col in enumerate(cols1):
        if col in list(tdf):
            tdf.rename(columns={col: cols2[i]}, inplace=True)

    tdf = tdf.filter(items=ls, axis=1)
    tdf['month_year'] = lab
    
    if lab == 'July_2023':
        gen_info = tdf.copy(deep=True)
    else:
        gen_info = gen_info.merge(tdf, how='outer')
        
    print(gen_info.shape)

print('filtering out hospitals not included in 2024 stars estimates and outcomes:')
hosps = stars_df_2024['PROVIDER_ID'].unique().tolist()
gen_info = gen_info[gen_info['PROVIDER_ID'].isin(hosps)]
print(gen_info.shape)

gen_info.head()

(5446, 8)
(10871, 8)
filtering out hospitals not included in 2024 stars estimates and outcomes:
(9356, 8)


,PROVIDER_ID,Hospital Name,State,2023 overall star rating,Hospital Type,Hospital Ownership,Emergency Services,month_year
0,010001,SOUTHEAST HEALTH MEDICAL CENTER,AL,3,Acute Care Hospitals,Government - Hospital District or Authority,Yes,July_2023
1,010005,MARSHALL MEDICAL CENTERS,AL,2,Acute Care Hospitals,Government - Hospital District or Authority,Yes,July_2023
2,010006,NORTH ALABAMA MEDICAL CENTER,AL,1,Acute Care Hospitals,Proprietary,Yes,July_2023
3,010007,MIZELL MEMORIAL HOSPITAL,AL,2,Acute Care Hospitals,Voluntary non-profit - Private,Yes,July_2023
4,010008,CRENSHAW COMMUNITY HOSPITAL,AL,Not Available,Acute Care Hospitals,Proprietary,Yes,July_2023


#      
------
# Merge estimated & actual stars outcomes with hospitals' general info

In [6]:
print(stars_df_2024.shape)
stars_df_2024 = stars_df_2024.merge(gen_info, how='outer')
print(stars_df_2024.shape)

stars_df_2024.head()

(9303, 28)
(9360, 34)


,PROVIDER_ID,Std_Outcomes_Mortality_score,Std_Outcomes_Readmission_score,Std_Outcomes_Safety_score,Std_PatientExp_score,Std_Process_score,std_weight_PatientExperience,std_weight_Readmission,std_weight_Mortality,std_weight_safety,std_weight_Process,weight_PatientExperience,weight_Outcomes_Readmission,weight_Outcomes_Mortality,weight_Outcomes_Safety,weight_Process,summary_score,Outcomes_Mortality_cnt,Outcomes_safety_cnt,Outcomes_Readmission_cnt,Patient_Experience_cnt,Process_cnt,Total_measure_group_cnt,MortSafe_Group_cnt,report_indicator,cnt_grp,star,month_year,Hospital Name,State,2023 overall star rating,Hospital Type,Hospital Ownership,Emergency Services
0,010001,0.007071,0.230867,-0.146867,0.137779,-0.677624,0.22,0.22,0.22,0.22,0.12,0.22,0.220000,0.22,0.22,0.120000,-0.030968,7.0,7.0,11.0,8.0,10.0,5.0,2.0,1.0,3) # of groups=5,3.0,Actual,SOUTHEAST HEALTH MEDICAL CENTER,AL,3,Acute Care Hospitals,Government - Hospital District or Authority,Yes
1,010005,-1.440587,0.720263,-0.087720,-0.255125,-0.489001,0.22,0.22,0.22,0.22,0.12,0.22,0.220000,0.22,0.22,0.120000,-0.292577,6.0,7.0,9.0,8.0,12.0,5.0,2.0,1.0,3) # of groups=5,2.0,Actual,MARSHALL MEDICAL CENTERS,AL,2,Acute Care Hospitals,Government - Hospital District or Authority,Yes
2,010006,-1.462748,-0.269475,-0.173331,-1.097088,-0.754912,0.22,0.22,0.22,0.22,0.12,0.22,0.220000,0.22,0.22,0.120000,-0.751171,7.0,7.0,9.0,8.0,11.0,5.0,2.0,1.0,3) # of groups=5,1.0,Actual,NORTH ALABAMA MEDICAL CENTER,AL,1,Acute Care Hospitals,Proprietary,Yes
3,010007,-3.527615,-0.470903,0.393999,0.199767,-1.804879,0.22,0.22,0.22,0.22,0.12,0.22,0.220000,0.22,0.22,0.120000,-0.965631,3.0,2.0,7.0,8.0,7.0,4.0,1.0,1.0,2) # of groups=4,1.0,Actual,MIZELL MEMORIAL HOSPITAL,AL,2,Acute Care Hospitals,Voluntary non-profit - Private,Yes
4,010008,NaN,-0.001726,NaN,NaN,-0.445200,0.22,0.22,0.22,0.22,0.12,NaN,0.647059,NaN,NaN,0.352941,-0.158246,0.0,0.0,2.0,0.0,6.0,1.0,0.0,0.0,NaN,NaN,Actual,CRENSHAW COMMUNITY HOSPITAL,AL,Not Available,Acute Care Hospitals,Proprietary,Yes


#      
------
# Find areas of accuracy


In [7]:
def get_stability_accuracy(sas_df, main_df):
    
    prvdrs1 = main_df['PROVIDER_ID'].unique()
    prvdrs2 = sas_df['PROVIDER_ID'].unique()
    
    ls = np.setdiff1d(list(prvdrs1), list(prvdrs2)).tolist()
    ls = np.setdiff1d(list(prvdrs2), list(prvdrs1)).tolist()
    
    tdf = sas_df[sas_df['star'].isin([1,2,3,4,5])]
    
    ls = list(set(prvdrs1) & set(prvdrs2))
    sas_df = sas_df[sas_df['PROVIDER_ID'].isin(ls)]
    sas_df.sort_values(by='PROVIDER_ID', ascending=False, inplace=True)
    main_df = main_df[main_df['PROVIDER_ID'].isin(ls)]
    main_df.sort_values(by='PROVIDER_ID', ascending=False, inplace=True)
    
    P1 = sas_df['PROVIDER_ID'].tolist()
    P2 = main_df['PROVIDER_ID'].tolist()
    
    for i, p1 in enumerate(P1):
        p2 = P2[i]
        if p1 != p2:
            print('P1 != P2')
            break
    
    
    pred_stars = main_df['star'].tolist()
    actual_stars = sas_df['star'].tolist()

    T_correct = 0
    T_incorrect = 0

    for i, p in enumerate(pred_stars):
        a = actual_stars[i]
        
        if np.isnan(a) and np.isnan(p):
            continue
            
        elif np.isnan(a):
            continue
        
        else:
            if p == a:
                T_correct += 1
            elif p != a:
                T_incorrect += 1

    print('    2024 STAR RATINGS:')
    print('    total correct:', T_correct)
    print('    total incorrect:', T_incorrect)
    print('    % correct:', np.round(100 * T_correct/(T_correct + T_incorrect), 3), '\n')
 

    stars = [1,2,3,4,5]
    for star in stars:
        tdf_sas = sas_df[sas_df['star'] == star]
        hosps = tdf_sas['PROVIDER_ID'].unique().tolist()
        tdf_main = main_df[main_df['PROVIDER_ID'].isin(hosps)]        
    
        tdf_sas.sort_values(by='PROVIDER_ID', ascending=False, inplace=True)
        tdf_main.sort_values(by='PROVIDER_ID', ascending=False, inplace=True)
    
        P1 = tdf_sas['PROVIDER_ID'].tolist()
        P2 = tdf_main['PROVIDER_ID'].tolist()

        for i, p1 in enumerate(P1):
            p2 = P2[i]
            if p1 != p2:
                print('P1 != P2')
                break
            
        pred_stars = tdf_main['star'].tolist()
        actual_stars = tdf_sas['star'].tolist()

        T_correct = 0
        T_incorrect = 0
        T = 0
        for i, p in enumerate(pred_stars):
            a = actual_stars[i]

            if np.isnan(a) and np.isnan(p):
                continue

            elif np.isnan(a):
                continue

            else:
                T += 1
                
                if p == a:
                    T_correct += 1
                elif p != a:
                    T_incorrect += 1
        
        if T > 0:
            print('        2024', star, 'hospitals:')
            print('        No. of actual stars:', T)
            #print('        No. of compare stars:', T_correct + T_incorrect)
            print('        total correct:', T_correct)
            print('        total incorrect:', T_incorrect)
            print('        % correct:', np.round(100 * T_correct/(T_correct + T_incorrect), 3), '\n')
        
    

In [8]:
print('Results for common providers\n')

cuts = ['2023 overall star rating']#, 'Hospital Type', 'Hospital Ownership']#, 'Emergency Services', 'State']
for cut in cuts:
    i_cuts = stars_df_2024[cut].unique().tolist()
    try:
        i_cuts.remove('Not Available')
    except:
        pass
    try:
        i_cuts.remove(np.nan)
    except:
        pass
    
    i_cuts.sort()
    
    for i_cut in i_cuts:
        est_df = stars_df_2024[stars_df_2024['month_year'] == 'July_2023']
        est_df = est_df[est_df['star'].isin([1,2,3,4,5])]
        est_df = est_df[est_df[cut] == i_cut]
        
        obs_df = stars_df_2024[stars_df_2024['month_year'] == 'Actual']
        obs_df = obs_df[obs_df['star'].isin([1,2,3,4,5])]
        obs_df = obs_df[obs_df[cut] == i_cut]
        
        if obs_df.shape[0] < 50 or est_df.shape[0] < 50:
            continue
        else:
            print(cut, ':', i_cut)        
            get_stability_accuracy(obs_df, est_df)



Results for common providers

2023 overall star rating : 1
    2024 STAR RATINGS:
    total correct: 170
    total incorrect: 55
    % correct: 75.556 

        2024 1 hospitals:
        No. of actual stars: 139
        total correct: 129
        total incorrect: 10
        % correct: 92.806 

        2024 2 hospitals:
        No. of actual stars: 71
        total correct: 38
        total incorrect: 33
        % correct: 53.521 

        2024 3 hospitals:
        No. of actual stars: 12
        total correct: 3
        total incorrect: 9
        % correct: 25.0 

        2024 4 hospitals:
        No. of actual stars: 3
        total correct: 0
        total incorrect: 3
        % correct: 0.0 

2023 overall star rating : 2
    2024 STAR RATINGS:
    total correct: 403
    total incorrect: 207
    % correct: 66.066 

        2024 1 hospitals:
        No. of actual stars: 116
        total correct: 70
        total incorrect: 46
        % correct: 60.345 

        2024 2 hospitals:
    